# Creating an Ethanol-Water Mixture

In this tutorial, we'll walk through creating a molecular system containing both water and ethanol molecules using `lammpsio`. We'll build a realistic mixture with proper molecular topologies, including bonds, angles, and dihedral angles. At the end, we will create a data file ready to be used by LAMMPS!

This tutorial demonstrates how to work with complex molecular systems that have different molecular topologies and shows how `lammpsio` can handle the creation of bonds, angles, and dihedrals for multiple molecular species.

First, we import the necessary packages.

In [13]:
import lammpsio
import numpy

## System Parameters

We need to define the composition of our mixture and read in the molecular coordinates. Our system will contain 125 water molecules and 125 ethanol molecules, creating an equal mixture.

Each water molecule has 3 atoms (1 oxygen, 2 hydrogen), and each ethanol molecule has 9 atoms (2 carbon, 1 oxygen, 6 hydrogen).

In [14]:
# System composition
N_water = 125
N_ethanol = 125

atoms_per_water = 3
atoms_per_ethanol = 9

## Generating the Molecular Configuration

Before we can build our LAMMPS data file, we need to create a realistic molecular configuration. We'll use [Packmol](http://leandro.iqm.unicamp.br/m3g/packmol/home.shtml), a powerful tool for creating initial configurations for molecular dynamics simulations by packing molecules in defined regions.

For this tutorial, we've provided the necessary input files for you to follow along:
- [`pack.inp`](pack.inp): Packmol input file that defines how to pack 125 water molecules and 125 ethanol molecules in a cubic box
- [`water.xyz`](water.xyz): Coordinates for a single water molecule
- [`ethanol.xyz`](ethanol.xyz): Coordinates for a single ethanol molecule

These files allow Packmol to generate our molecular coordinates. Let's run Packmol to create the mixed system.

In [15]:
import os

# Execute packmol
os.system('packmol < pack.inp >/dev/null 2>&1');

## Reading Coordinates

Now we'll read the molecular coordinates from the mixture.xyz file generated by Packmol and separate them into water and ethanol positions. Packmol writes coordinates in the order specified in the input, so water molecules appear first and ethanol molecules second.

In [16]:
positions = numpy.genfromtxt('mixture.xyz', skip_header=2, dtype=numpy.float64)[:, 1:4]


pos_water = positions[:N_water * atoms_per_water]
pos_ethanol = positions[N_water * atoms_per_water:]

## Creating the Snapshot

Now we'll create a `Snapshot` to hold all our molecular data. We need to calculate the total number of atoms and define a simulation box that contains our system. We'll use the same box dimensions that were specified in our Packmol input file: a cubic box extending from -12.5 to 12.5 Å in each direction.

In [17]:
N_total = N_water * atoms_per_water + N_ethanol * atoms_per_ethanol
box = lammpsio.Box([-12.5, -12.5, -12.5], [12.5, 12.5, 12.5])
snap = lammpsio.Snapshot(N_total, box, step=10)

## Assigning Atomic Properties

Now we need to assign positions, masses, and type IDs to each atom. We have three atom types:
- Type 1: Hydrogen (H) with mass 1
- Type 2: Oxygen (O) with mass 16
- Type 3: Carbon (C) with mass 12

For water molecules, the atom order is O-H-H, and for ethanol molecules, the atom order is H-C-H-C-O-H-H-H-H.

In [18]:
# Assign positions
snap.position[:N_water * atoms_per_water] = pos_water
snap.position[N_water * atoms_per_water:] = pos_ethanol

# Assign masses: type 1 (H)=1.0, type 2 (O)=16.0, type 3 (C)=12.0
snap.mass[:N_water * atoms_per_water] = numpy.tile([16.0, 1.0, 1.0], N_water)
snap.mass[N_water * atoms_per_water:] = numpy.tile(
    [1.0, 12.0, 1.0, 12.0, 16.0, 1.0, 1.0, 1.0, 1.0], N_ethanol
    )

# Assign type IDs
snap.typeid[:N_water * atoms_per_water] = numpy.tile([2, 1, 1], N_water)
snap.typeid[N_water * atoms_per_water:] = numpy.tile(
    [1, 3, 1, 3, 2, 1, 1, 1, 1], N_ethanol
    )

## Creating Bonds

Molecular systems require bond information to define which atoms are chemically bonded. For this tutorial, we'll define bonded interactions purely by atom type (H, O, C) to keep the example simple. In general, however, atom typing — and therefore the assignment of bond, angle, and dihedral parameters — depends on the force field you choose; `lammpsio` does not perform force-field-specific atom typing or parameter assignment.

Our bond types are: 
- Bond type 1: C-C bonds
- Bond type 2: C-H bonds  
- Bond type 3: O-H bonds
- Bond type 4: C-O bonds

### Creating Water Bonds

We'll start with the simpler water molecules. Each water molecule has 2 bonds connecting the oxygen to each hydrogen. We can define a bond template that describes the connectivity within a single water molecule, then replicate this pattern for all water molecules in our system.

In [19]:
bond_members = []
bond_typeid = []

water_bond_template = [[0, 1],  
                       [0, 2]]

# Water molecules: each has 2 O-H bonds (type 3)
for i in range(N_water):
    for bond in water_bond_template:
        bond_members.append([i*atoms_per_water + bond[0], i*atoms_per_water + bond[1]])
    bond_typeid.extend([3, 3])  # Both O-H bonds

### Creating Ethanol Bonds

Ethanol molecules are more complex, with 8 bonds per molecule connecting various atoms. We use the same template approach: define the bond pattern for a single ethanol molecule, then replicate it for all ethanol molecules. The template describes which atoms within each molecule are bonded and what type each bond should be.

In [20]:
# Ethanol molecules
ethanol_bond_template = [[0, 1],  # H-C1 (bond type 2)
                        [1, 2],  # C1-H (bond type 2)
                        [1, 3],  # C1-C2 (bond type 1)
                        [1, 8],  # C1-H (bond type 2)
                        [3, 4],  # C2-O (bond type 4)
                        [3, 5],  # C2-H (bond type 2)
                        [3, 7],  # C2-H (bond type 2)
                        [4, 6]]  # O-H (bond type 3)

ethanol_bond_type_template = [2, 2, 1, 2, 4, 2, 2, 3]

for i in range(N_ethanol):
    # Offset atoms by: water atoms + atoms from previous ethanol molecules
    offset = N_water * atoms_per_water + i * atoms_per_ethanol
    for bond in ethanol_bond_template:
        bond_members.append([offset + bond[0], offset + bond[1]])
    bond_typeid.extend(ethanol_bond_type_template)

### Combining All Bonds

Now we use the bond information we've collected to create the `Bonds` object in our snapshot. This object will contain all the bond connectivity and type information that LAMMPS needs. Note that LAMMPS uses 1-based indexing for atom IDs, so we add 1 to all our 0-based Python indices.

In [21]:
N_bonds = len(bond_members)
snap.bonds = lammpsio.Bonds(N=N_bonds, num_types=4)
snap.bonds.members = numpy.array(bond_members) + 1 # Adding 1 to match LAMMPS indexing
snap.bonds.typeid = numpy.array(bond_typeid)

## Creating Angles

We follow the same template approach we used for bonds: define the angle pattern within each molecule type, then replicate it across all molecules of that type.

Our angle types are: 
- Type 1: H-O-H
- Type 2: H-C-H
- Type 3: H-C-C
- Type 4: C-C-O
- Type 5: C-O-H
- Type 6: H-C-O

Water molecules are simple with just one angle per molecule (H-O-H), while ethanol molecules have multiple angles around their carbon and oxygen centers.

In [22]:
angle_members = []
angle_typeid = []

# Water molecules
for i in range(N_water):
    atom_start = i * 3 
    angle_members.append([atom_start + 1, atom_start, atom_start + 2])  # H-O-H
    angle_typeid.append(1)

# Ethanol molecules
ethanol_angle_template = [[0, 1, 2],  # H-C1-H (type 2)
                        [0, 1, 3],  # H-C1-C2 (type 3)
                        [0, 1, 8],  # H-C1-H (type 2)
                        [2, 1, 3],  # H-C1-C2 (type 3)
                        [2, 1, 8],  # H-C1-H (type 2)
                        [3, 1, 8],  # C2-C1-H (type 3)
                        [1, 3, 4],  # C1-C2-O (type 4)
                        [1, 3, 5],  # C1-C2-H (type 3)
                        [1, 3, 7],  # C1-C2-H (type 3)
                        [4, 3, 5],  # O-C2-H (type 6)
                        [4, 3, 7],  # O-C2-H (type 6)
                        [5, 3, 7],  # H-C2-H (type 2)
                        [3, 4, 6]]  # C2-O-H (type 5)

ethanol_angle_typeid_template = [2, 3, 2, 3, 2, 3, 4, 3, 3, 6, 6, 2, 5]

for i in range(N_ethanol):
    offset = N_water * atoms_per_water + i * atoms_per_ethanol
    for angle in ethanol_angle_template:
        angle_members.append([offset + angle[0], 
                                    offset + angle[1], 
                                    offset + angle[2]])
    angle_typeid.extend(ethanol_angle_typeid_template)

# Create the angles object  
N_angles = len(angle_members)
snap.angles = lammpsio.Angles(N=N_angles, num_types=6)
snap.angles.members = numpy.array(angle_members)+1 # Adding 1 to match LAMMPS indexing
snap.angles.typeid = numpy.array(angle_typeid)

## Creating Dihedral Angles

For ethanol molecules, we also need dihedral angles (4-atom torsional angles) to properly define the molecular geometry. Water molecules don't have dihedral angles since they only have 3 atoms.

We'll define dihedral types as:
- Type 1: H-C-C-H 
- Type 2: H-C-C-O  
- Type 3: C-C-O-H

In [23]:
dihedral_members = []
dihedral_types = []

dihedral_template = [[0, 1, 3, 4],   # H-C-C-O (type 2)
                    [0, 1, 3, 5],    # H-C-C-H (type 1)
                    [0, 1, 3, 7],    # H-C-C-H (type 1)
                    [2, 1, 3, 4],    # H-C-C-O (type 2)
                    [2, 1, 3, 5],    # H-C-C-H (type 1)
                    [2, 1, 3, 7],    # H-C-C-H (type 1)
                    [8, 1, 3, 4],    # H-C-C-O (type 2)
                    [8, 1, 3, 5],    # H-C-C-H (type 1)
                    [8, 1, 3, 7],    # H-C-C-H (type 1)
                    [1, 3, 4, 6]]    # C-C-O-H (type 3)
 
dihedral_type_template = [2, 1, 1, 2, 1, 1, 2, 1, 1, 3]

# Only ethanol molecules have dihedrals
for i in range(N_ethanol):
    offset = N_water * atoms_per_water + i * atoms_per_ethanol
    for j, dihedral in enumerate(dihedral_template):
        dihedral_members.append([offset + dihedral[0], offset + dihedral[1], 
                                       offset + dihedral[2], offset + dihedral[3]])
    dihedral_types.extend(dihedral_type_template)

dihedral_members = numpy.array(dihedral_members)

# Create the dihedrals object
N_dihedrals = len(dihedral_members)
snap.dihedrals = lammpsio.Dihedrals(N=N_dihedrals, num_types=3)
snap.dihedrals.members = numpy.array(dihedral_members)+1 # Adding 1 to match LAMMPS indexing
snap.dihedrals.typeid = numpy.array(dihedral_types)

## Save to LAMMPS Data File

Finally, we save the complete molecular system to a LAMMPS data file!

In [24]:
# Write to LAMMPS data file
lammpsio.DataFile.create(
    filename="ethanol_water_mixture.data", snapshot=snap
)

## Summary

In this tutorial, we created an ethanol-water mixture using `lammpsio`. We demonstrated how to work with molecular systems that require bonds, angles, and dihedral angles. The key steps were:

1. **Generating coordinates** using Packmol to create a realistic mixture
2. **Creating molecular topology** by defining bonds, angles, and dihedrals based on atom types
3. **Saving to LAMMPS format** ready for molecular dynamics simulations

This approach shows how `lammpsio` can handle complex molecular systems with multiple species. The resulting data file includes all particle and topological information required to run a simulation of ethanol–water mixtures when paired with an appropriate force field.